In [1]:
import wandb
import torch
api = wandb.Api()

In [2]:
import re
import os
import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_theme()
# sns.set_style('whitegrid')
from pprint import pprint


In [3]:
def my_groupby(collection, extractor):
    res = {}
    for item in collection:
        key = extractor(item)
        if key not in res:
            res[key] = []
        res[key].append(item)
    return res


In [4]:
fever_exps = api.runs('redaigbaria/fever_exps')

In [5]:
# filtered_runs = fever_exps
# filtered_runs = list(filter(lambda r: 'bert_UNB_ConfReg_32' == r.name, filtered_runs))
# for r in filtered_runs:
#     print(r.name)
#     r.group = r.name
#     r.update()
# print(filtered_runs)


In [6]:
project_base='/home/redaigbaria/research_project/examples/pytorch/text-classification/'
runs = api.runs("redaigbaria/mnli_exps")
probing_runs = api.runs("redaigbaria/online-code")

In [7]:
# # Compute compression statstics

# filtered_probing_runs = list(filter(lambda r: r.config['task_name'] == 'mnli-lex-class-sub', probing_runs))
# grouped_probing_runs = my_groupby(filtered_probing_runs, lambda r: r.group)

# for k in grouped_probing_runs:
#     tmp_res = []
#     try:
#         for r in grouped_probing_runs[k]:
#             tmp_res.append(r.summary['compression'])
#         tmp_res = torch.tensor(tmp_res)
#         print(f"{k}: $ %2.2f \pm %1.1f $" % (tmp_res.mean(), tmp_res.std(unbiased=False)))
#     except:
#         print(f"{k} failed")


In [8]:
# for run in runs:
#     # and re.match("tinybert_\d", run.name)
#     if run.group != run.name:
#         # if re.match("bert_\d_claim_only", run.name):
#         #     run.group = "bert_claim_only"
#         #     run.update()
#         print(f"{run}, {run.name}, {run.group}")
#         # run.group = run.name
#         # run.update()
#         # run.group = "tinybert"
        
#         # run.update()

In [9]:
filtered_runs = runs

# filtered_runs = list(filter(lambda r: 'enforce_similarity' in r.config and r.config['enforce_similarity'], filtered_runs))
filtered_runs = list(filter(lambda r: r.group is not None and r.state == 'finished', filtered_runs))
filtered_runs = list(filter(lambda r: 'deberta' not in r.group, filtered_runs))
# filtered_runs = list(filter(lambda r: os.path.isdir(os.path.join(project_base, r.config['output_dir'])), filtered_runs))
filtered_runs = list(filter(lambda r: 'lexical_bias' in r.tags, filtered_runs))
# filtered_runs = list(filter(lambda r: r.config['weak_model_path'] != 'None', filtered_runs))
# filtered_runs = list(filter(lambda r: r.name == "bert_EDBXE_STOB_lexicalDeBias_abs_cos_cor_9M-10M-11M_9-10-11_lambda_1_100_64_sim", filtered_runs))

In [10]:
grouped_runs = my_groupby(filtered_runs, lambda r: r.group)
probing_grouped_runs = my_groupby(probing_runs, lambda r: r.group)

In [ ]:
failed_runs = set()
compression_hist = {}
proper_formatting = {
    # sim
    True: {
        # grad
        True: [],
        False: []
    },
    False: {
        # grad
        True: [],
        False: []
    }
}
for k, current_group in grouped_runs.items():
    if k == "bert_LexicalModel_lexicalDeBias_POE_64":
        overlap_key = "mnli-lex-class/bert_64_POE"
    else:
        overlap_key = f"mnli-lex-class/{k}"
    sub_key = f"mnli-lex-class-sub/{k}"
    if overlap_key not in probing_grouped_runs.keys():
        print(f"{k} failed2!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        failed_runs.add(k)
        continue
    # if sub_key not in probing_grouped_runs.keys():
    #     failed_runs.add(k)
    #     print(f"{k} failed1!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    #     continue
    sim = current_group[0].config['enforce_similarity']
    GReg = current_group[0].config['regularize_grads']
    sim_arrow = "$\\uparrow$" if sim else "$\downarrow$"
    GReg_letter = "$\\nabla$ " if GReg else ""
    # compute HANS-non_ent
    lex_res = []
    sub_res = []
    for r in current_group:
        hans_path = f"{r.config['output_dir']}/hans/evaluate_heuristic_output.txt"
        if os.path.isfile(hans_path):
            run_hans_file = open(hans_path, "r")
            run_hans = run_hans_file.readlines()
            lex_res.append(float(run_hans[6].split(":")[1].strip()))
            sub_res.append(float(run_hans[7].split(":")[1].strip()))
            run_hans_file.close()
        # res.append(r.summary['eval/HANS_non_ent_accuracy'])
    lex_res = torch.tensor(lex_res) * 100
    sub_res = torch.tensor(sub_res) * 100
    lex_mean, lex_std = lex_res.mean().item(), lex_res.std().item()
    sub_mean, sub_std = sub_res.mean().item(), sub_res.std().item()
    # lex-compression
    res = []
    for r in probing_grouped_runs[overlap_key]:
        res.append(r.summary['compression'])
    res = torch.tensor(res)
    lex_compression_mean, lex_compression_std = res.mean().item(), res.std().item()
    # sub-compression
    res = []
    if sub_key not in probing_grouped_runs.keys():
        sub_compression_mean, sub_compression_std = -1, -1
    else:
        for r in probing_grouped_runs[sub_key]:
            try:
                res.append(r.summary['compression'])
            except Exception as e:
                print(e)
        res = torch.tensor(res).float()
        sub_compression_mean, sub_compression_std = res.mean().item(), res.std().item()
    
    compression_hist[k] = (sub_mean, sub_compression_mean, sub_compression_std, lex_compression_mean, lex_compression_std, lex_mean)
    clean_name = k.replace("_GReg", "").replace("_sim", "").replace("_STOB", "").replace("bert_", "").replace("_lexicalDeBias", "").replace("lexicalDeBias", "").replace("_64", "").replace("_lambda", "").replace("EXDBE_", "").replace("EDBXE_", "").replace('_', '-')
    clean_name = re.sub('-(\d)-(\d+)$', ' $(lambda=\\2)$', clean_name).replace("lambda", "\\lambda")
    row = f"{GReg_letter}\\small{{{clean_name}}} {sim_arrow} & $ %.1f \\pm %.1f $ & $ %2.2f $ & $ %.2f \\pm %.1f $ & $ %.2f $ \\\\" % (lex_compression_mean, lex_compression_std, lex_mean, sub_compression_mean, sub_compression_std, sub_mean)
    proper_formatting[sim][GReg].append(row)

for g_sim in proper_formatting.values():
    for grad_g in g_sim.values():
        for row in grad_g:
            print(row)


In [12]:
relevant_points = [
    # (name, sub_x_translation, sub_y_translation, overlap_x_translation, overlap_y_translation)
    ('bert_EDBXE_STOB_lexicalDeBias_linear_cka_9M-10M-11M_9-10-11_lambda_1_100_64_sim',-6, 0, 0.5, 0),
    ('bert_EDBXE_STOB_lexicalDeBias_cos_cor_eM-0M-6M-10M-11M_e-0-6-10-11_lambda_1_10_64_sim_GReg',0.5, -0.05, -1.7, -0.4),
    ('bert_LexicalModel_STOB_lexicalDeBias_abs_cos_cor_eM_2_lambda_1_10_64_GReg',0.5, 0, 0, 0.1),
    # ('bert_LexicalModel_STOB_lexicalDeBias_abs_cos_cor_eM-0M-6M-11M_2-2-2-2-2_lambda_1_10_64',0, 0, 0, 0),
    ('bert_UB2_STOB_lexicalDeBias_abs_cos_cor_eM-0M-6M-11M_e-0-6-10-11_lambda_1_10_64',-0.6, -0.25, -1.5, 0.1),
    ('bert_LexicalModel_lexicalDeBias_POE_64',-1.5, -0.3, -1.3, -0.3),
    ('conf_reg',-6, 0, -5.5, 0),
    ('bert_64_base', 0.7, 0.05, 1, 0.1)
]
def plot_points(relevant_points):
    labels = [
        "$SimReg\\uparrow $",
        '$\\nabla SimReg\\uparrow$',
        '$\\nabla SimReg\\downarrow$',
        '$SimReg\\downarrow$',
        # 'UB $\\downarrow$',
        '$POE$',
        '$ConfReg$',
        '$baseline$'
    ]
    # sns.set_style('darkgrid')
    # sns.set_context('paper')


    from tueplots import axes, bundles
    plt.clf()
    with plt.rc_context(bundles.iclr2023()):
        plt.rcParams.update({
            'figure.dpi': 200,
            'font.size': 12,
            'axes.titlesize': 16,
            'legend.fontsize': 14,
            'axes.labelsize': 14,
            'xtick.labelsize': 10,
            'ytick.labelsize': 10,
        })
        fig, ax = plt.subplots()
        # sub_mean, sub_compression_mean, sub_compression_std, lex_compression_mean, lex_compression_std, lex_mean
        compression_hist['conf_reg'] = (60.89, 5.81, 0.5608, 6.298, 0.7, 6.6, 0)
        compression_axs = 3

        baseline_compression = compression_hist['bert_64_base'][compression_axs]
        robustnesses = {
            1: [],
            3: []
        }
        compression_means = []
        compression_std = []
        for point in relevant_points:
            k = point[0]
            current_run = compression_hist[k]
            
            robustnesses[1].append(current_run[0] - compression_hist['bert_64_base'][0])
            robustnesses[3].append(current_run[-1] - compression_hist['bert_64_base'][-1])
            
            compression_means.append(current_run[compression_axs] - baseline_compression)
            compression_std.append(current_run[4])

        ax.errorbar(robustnesses[compression_axs][:-1], compression_means[:-1], yerr=compression_std[:-1], fmt="s")
        ax.errorbar(robustnesses[compression_axs][-1:],compression_means[-1:], yerr=compression_std[-1:], fmt="o")
        for i, point in enumerate(relevant_points):
            k = point[0]
            current_run =  compression_hist[k]
            ax.annotate(labels[i], (robustnesses[compression_axs][i] + point[compression_axs], compression_means[i] + point[compression_axs + 1]))
        ax.set_xlabel('$ \\Delta$ HANS')
        ax.set_ylabel('$ \\Delta$ Compression')
        bias_type = 'Overlap' if compression_axs == 3 else 'Subsequence'
        ax.set_title(bias_type)
        print(bias_type)
        ax.grid()
        plt.show()
        # plt.savefig(f'{bias_type.lower()}_compression_analysis.pdf')

plot_points(relevant_points)

# for p in relevant_points:
#     if p[0] in grouped_runs:
#         for r in grouped_runs[p[0]]:
#             print(f"""
#                 sbatch --gres=gpu:1 -w plato1 ./hans_model.sh {r.config['output_dir']}
#             """)

Overlap


<Figure size 432x288 with 0 Axes>

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1100x679.837 with 1 Axes>

In [ ]:
total_runs = 0
for k, v in grouped_runs.items():
    # for r in v:
    print(f"{k}: {len(v)}")
    total_runs += len(v)

In [175]:
dispatched_jobs = set()
saved=0
root_dir="/home/redaigbaria/research_project/examples/pytorch/text-classification"
for k in grouped_runs.keys():
    # if len(grouped_runs[k]) == 1:
    #     continue
    if f"mnli-lex-class-sub/{k}" in probing_grouped_runs.keys():
        continue
    for run in grouped_runs[k]:
        model_dir = os.path.join(root_dir, run.config['output_dir'])
        dispatched_jobs.add(f"nlp_sbatch plato1 probing.py --model_name_or_path {model_dir} --name {k} --task_config_file mnli_lex_class.json --seed 42\n")
        # dispatched_jobs.add(f"nlp_sbatch plato1 probing.py --model_name_or_path {model_dir} --name {k} --task_config_file mnli_lex_class_sub.json --seed 42\n")

print(len(dispatched_jobs))


13


In [ ]:
for j in dispatched_jobs:
    print(j)

In [ ]:
hist = run.history()

In [ ]:
plot_data = hist[['eval/HANS_non_ent_accuracy', 'train/global_step']].dropna()

In [ ]:
plt.plot(plot_data['train/global_step'], plot_data['eval/HANS_non_ent_accuracy'])

In [ ]:
def prepare_row(runs, metric):
    [r.summary()[metric]    for r in runs]